In [1]:
import h5py
import numpy as np
from analysis_tools_2x2 import visualize
import importlib
import matplotlib.pyplot as plt

In [25]:
# open files, get datasets
larndsim_file = '/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/2x2/MC/larndsim/MiniRun3_1E19_RHC.larnd_v2.00001.LARNDSIM.h5'
clustered_charge_file = '/sdf/group/neutrino/sfogarty/ND_prototype_files/charge_data/2x2/MC/clusters/MiniRun3_1E19_RHC.larnd_v2.00001.LARNDSIM_clusters.h5'

h5 = h5py.File(larndsim_file,'r')
packets = h5['packets']
tracks = h5['tracks']

trajectories = h5['trajectories']
h5_clusters = h5py.File(clustered_charge_file,'r')
clusters = h5_clusters['clusters']
hits = h5_clusters['clusters_hits']

genie_hdr = h5['genie_hdr']
genie_stack = h5['genie_stack']
light_trig = h5['light_trig']
light_wvfm = h5['light_wvfm']

target_mask = (genie_hdr['target'] == 18)
position_mask_x = np.abs(genie_hdr['vertex'][:,0]*10) < 655
position_mask_y = np.abs(genie_hdr['vertex'][:,1]*10 - 420) < 655
position_mask_z = np.abs(genie_hdr['vertex'][:,2]*10) < 655
genie_hdr_argon = genie_hdr[target_mask & position_mask_x & position_mask_y & position_mask_z]

In [5]:
h5.keys()

<KeysViewHDF5 ['_header', 'configs', 'genie_hdr', 'genie_stack', 'light_dat', 'light_trig', 'light_wvfm', 'mc_packets_assn', 'messages', 'packets', 'tracks', 'trajectories', 'vertices']>

In [7]:
h5['trajectories'].dtype

dtype({'names':['eventID','vertexID','trackID','local_trackID','parentID','E_start','pxyz_start','xyz_start','t_start','E_end','pxyz_end','xyz_end','t_end','pdgId','start_process','start_subprocess','end_process','end_subprocess'], 'formats':['<u4','<u8','<u4','<u4','<i4','<f4',('<f4', (3,)),('<f4', (3,)),'<f4','<f4',('<f4', (3,)),('<f4', (3,)),'<f4','<i4','<u4','<u4','<u4','<u4'], 'offsets':[0,8,16,20,24,28,32,44,56,60,64,76,88,92,96,100,104,108], 'itemsize':112})

In [33]:
all_trajectories = np.array(h5['trajectories'])
proton_trajectories = all_trajectories[all_trajectories['pdgId'] == 2212]
neutron_trajectories = all_trajectories[all_trajectories['pdgId'] == 2112]

position_mask_x = (np.abs(proton_trajectories['xyz_start'][:,0]*10) < 625) & (np.abs(proton_trajectories['xyz_end'][:,0]*10) < 625)
position_mask_y = (np.abs(proton_trajectories['xyz_start'][:,1]*10 - 420) < 625) & (np.abs(proton_trajectories['xyz_end'][:,1]*10 - 420) < 625)
position_mask_z = (np.abs(proton_trajectories['xyz_start'][:,2]*10) < 625) & (np.abs(proton_trajectories['xyz_end'][:,2]*10) < 625)
proton_trajectories = proton_trajectories[position_mask_x & position_mask_y & position_mask_z]

position_mask_x = (np.abs(neutron_trajectories['xyz_start'][:,0]*10) < 625) & (np.abs(neutron_trajectories['xyz_end'][:,0]*10) < 625)
position_mask_y = (np.abs(neutron_trajectories['xyz_start'][:,1]*10 - 420) < 625) & (np.abs(neutron_trajectories['xyz_end'][:,1]*10 - 420) < 625)
position_mask_z = (np.abs(neutron_trajectories['xyz_start'][:,2]*10) < 625) & (np.abs(neutron_trajectories['xyz_end'][:,2]*10) < 625)
neutron_trajectories = neutron_trajectories[position_mask_x & position_mask_y & position_mask_z]

proton_start_processes = proton_trajectories['start_process']*1000 + proton_trajectories['start_subprocess']
print(list(zip(np.unique(proton_start_processes, return_counts=True)[0],np.unique(proton_start_processes, return_counts=True)[1])))

[(1091, 7), (2002, 16), (2003, 3), (2013, 7), (2022, 11), (4111, 1), (4121, 206), (4151, 4), (6201, 1)]


In [43]:
from tqdm import tqdm
nu_pdg = [-12,12,-14,14]
proton_trackIDs = []
neutron_length_cut_cm = 5
proton_length_cut_cm = 0.5
for i,parentID in tqdm(enumerate(proton_trajectories['parentID'])):
    
    if parentID != -1:
        parentTrajectory = neutron_trajectories[neutron_trajectories['trackID'] == parentID]
        if len(parentTrajectory) > 0:
            d_neutron_cm = np.sqrt(np.sum((parentTrajectory['xyz_start'] - parentTrajectory['xyz_end'])**2))
            d_proton_cm = np.sqrt(np.sum((proton_trajectories[i]['xyz_start'] - proton_trajectories[i]['xyz_end'])**2))

            if d_neutron_cm > neutron_length_cut_cm and d_proton_cm > proton_length_cut_cm:
                proton_trackIDs.append(i)
            
protons_from_neutrons_trajectories = proton_trajectories[proton_trackIDs]


256it [00:00, 16790.86it/s]


In [44]:
np.unique(protons_from_neutrons_trajectories['eventID'])

array([1016, 1021, 1034, 1045, 1048, 1096, 1108, 1109, 1134, 1140, 1168,
       1169, 1178, 1183], dtype=uint32)

In [24]:
protons_from_neutrons_trajectories_dcut['eventID']

array([1005, 1008, 1013, 1016, 1018, 1021, 1025, 1030, 1030, 1031, 1032,
       1034, 1034, 1037, 1037, 1048, 1048, 1048, 1048, 1048, 1054, 1056,
       1058, 1073, 1073, 1073, 1078, 1078, 1079, 1079, 1079, 1079, 1082,
       1082, 1082, 1082, 1092, 1094, 1095, 1096, 1096, 1100, 1105, 1105,
       1107, 1107, 1108, 1109, 1112, 1118, 1121, 1126, 1126, 1128, 1134,
       1138, 1139, 1149, 1151, 1153, 1154, 1155, 1155, 1155, 1159, 1160,
       1163, 1163, 1163, 1163, 1163, 1169, 1170, 1170, 1178, 1178, 1183,
       1184, 1184, 1187, 1187, 1193, 1194], dtype=uint32)

In [43]:
proton_trajectories_event['trackID']

array([2544, 2869, 2881, 2902, 2907, 3302], dtype=uint32)

In [51]:
all_trajectories[all_trajectories['trackID'] == 840182][0]['eventID']

8199

In [76]:
grandparentTrajectory

array([],
      dtype={'names':['eventID','vertexID','trackID','local_trackID','parentID','E_start','pxyz_start','xyz_start','t_start','E_end','pxyz_end','xyz_end','t_end','pdgId','start_process','start_subprocess','end_process','end_subprocess'], 'formats':['<u4','<u8','<u4','<u4','<i4','<f4',('<f4', (3,)),('<f4', (3,)),'<f4','<f4',('<f4', (3,)),('<f4', (3,)),'<f4','<i4','<u4','<u4','<u4','<u4'], 'offsets':[0,8,16,20,24,28,32,44,56,60,64,76,88,92,96,100,104,108], 'itemsize':112})

In [102]:
np.sum(all_trajectories['pdgId'] == -16)

0

In [9]:
np.sum(all_trajectories[all_trajectories['eventID'] == 1134]['pdgId'] == 14)

0

In [11]:
all_trajectories['vertexID'][0:100]

array([1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021, 1000000010000021, 1000000010000021,
       1000000010000021,